In [1]:
import os 
os.chdir("../..")

In [2]:
import sys
os.path.dirname(sys.executable)

'/home/h/anaconda3/envs/geopython37/bin'

In [3]:
import numpy as np
import json
from detectron2.structures import BoxMode
import pandas as pd
import geopandas as gpd
import cv2
import matplotlib.pyplot as plt


from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from src.utils.dataset import split_train_val_test, get_detectron2_dataset_dicts, register_detectron2_datasets
import pickle

In [4]:
if os.path.exists("dsd"):
    with open("/home/h//szpdsdstion.pkl", "rb") as f:
        splits = pickle.load(f)
else:
    samples = pd.read_pickle("/home/h//szprotawa_patches_256_rgb_ir/annotation.pkl")["patch_number"].unique()
    splits = split_train_val_test(samples, 0.7, 0.15, 0.15)
    with open("/home/h//szprotawa_patches_256_rgb_ir/szprotawa_splits.pkl", "wb") as f:
        pickle.dump(splits, f)


register_detectron2_datasets("szprotawa", "/home/h//szprotawa_patches_256_rgb_ir", 
                                      splits, min_bbox_area=200, 
                                      limits={"train": -1, "val": -1, "test": -1})

In [5]:
!ls /home/h//szprotawa_patches_256_rgb_ir

h_14369.tif  patch_19085.tif  patch_5007.tif   patch_9711.tif
patch_14370.tif  patch_19086.tif  patch_5008.tif   patch_9712.tif
patch_14371.tif  patch_19087.tif  patch_5009.tif   patch_9713.tif
patch_14372.tif  patch_19088.tif  patch_5010.tif   patch_9714.tif
patch_14373.tif  patch_19089.tif  patch_5011.tif   patch_9715.tif
patch_14374.tif  patch_1908.tif   patch_5012.tif   patch_9716.tif
patch_14375.tif  patch_19090.tif  patch_5013.tif   patch_9717.tif
patch_14376.tif  patch_19091.tif  patch_5014.tif   patch_9718.tif
patch_14377.tif  patch_19092.tif  patch_5015.tif   patch_9719.tif
patch_14378.tif  patch_19093.tif  patch_5016.tif   patch_971.tif
patch_14379.tif  patch_19094.tif  patch_5017.tif   patch_9720.tif
patch_14380.tif  patch_19095.tif  patch_5018.tif   patch_9721.tif
patch_14381.tif  patch_19096.tif  patch_5019.tif   patch_9722.tif
patch_14382.tif  patch_19097.tif  patch_5020.tif   patch_9723.tif
patch_14383.tif  patch_19098.tif  patch_5021.tif   patch_9724.tif
patch_14384.tif

In [6]:
meta = MetadataCatalog.get("szprotawa_train")
meta

Metadata(name='szprotawa_train', thing_classes=['SickTrees'])

In [7]:
dataset_dicts = DatasetCatalog.get("szprotawa_train")
print(len(dataset_dicts))
dataset_dicts = DatasetCatalog.get("szprotawa_test")
print(len(dataset_dicts))
dataset_dicts = DatasetCatalog.get("szprotawa_val")
print(len(dataset_dicts))

1024
226
221


In [8]:
'''
d = get_detectron2_dataset_dicts("/home/h/szprotawa_patches_256_rgb_ir/", splits['val'],200,100)
len(d)
d[0]
'''

'\nd = get_detectron2_dataset_dicts("/home/h/szprotawa_patches_256_rgb_ir/", splits[\'val\'],200,100)\nlen(d)\nd[0]\n'

In [9]:
splits.keys()

dict_keys(['train', 'val', 'test'])

smpl = np.random.choice(dataset_dicts, 1)
for d in smpl:
    print(d["file_name"])
    img = cv2.imread(d["file_name"])[:, :, ::-1]
    visualizer = Visualizer(img, metadata=meta, scale=0.75)
    vis = visualizer.draw_dataset_dict(d)
    plt.figure()
    plt.imshow(vis.get_image())

In [10]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir tboard_logs

In [11]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

In [12]:
from datetime import datetime

In [13]:
cfg = get_cfg()

cfg.OUTPUT_DIR = "tboard_logs/retinanet_test_2020-01-21T23:40/"
# cfg.OUTPUT_DIR = f"./tboard_logs/retinanet_test_{datetime.now().isoformat().rsplit(':', 1)[0]}/"
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("szprotawa_train",)
#cfg.DATASETS.TEST = ("szprotawa_val", )
#cfg.DATASETS.VAL = ("szprotawa_val", )
cfg.DATALOADER.NUM_WORKERS = 4

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0001  # pick a good LR
cfg.SOLVER.MAX_ITER = 30_000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.SOLVER.CHECKPOINT_PERIOD = 250


# cfg.MODEL.RETINANET.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.RETINANET.NUM_CLASSES = 1  # only has one class (ballon)

cfg.TEST.EVAL_PERIOD = 50000 #200
cfg.TEST.DETECTIONS_PER_IMAGE = 100

cfg.VIS_PERIOD = 100
cfg.MODEL.PIXEL_MEAN = [103.53, 116.28, 123.675, 100.1] #ostatnia wartość wpisana z palca
cfg.MODEL.PIXEL_STD =  [1.0, 1.0, 1.0, 1.0] #ostatnia wartość wpisana z palca

Loading config /home/h/anaconda3/envs/geopython37/lib/python3.7/site-packages/detectron2-0.1-py3.7-linux-x86_64.egg/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [14]:
print(cfg)

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('coco_2017_val',)
  TRAIN: ('szprotawa_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32, 40.31747359663594, 50.79683366298238], [64, 80.63494719327188, 101.59366732596476], [128, 161.26989438654377, 203.18733465192952], [256, 322.53978877308754, 406.37466930385904], [512, 64

In [15]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader


class Trainer(DefaultTrainer):
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return COCOEvaluator(dataset_name, cfg, False, output_dir=cfg.OUTPUT_DIR+f"/eval/{dataset_name}")


In [27]:
with open(cfg.OUTPUT_DIR + "/config.yml", "w") as f:
    f.write(cfg.dump())

In [16]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

1  data_time: 0.0069  lr: 0.000100  max_mem: 2537M
[02/13 10:05:20 d2.utils.events]: eta: 0:45:31  iter: 22559  total_loss: 0.375  loss_cls: 0.285  loss_box_reg: 0.111  time: 0.3661  data_time: 0.0068  lr: 0.000100  max_mem: 2537M
[02/13 10:05:27 d2.utils.events]: eta: 0:45:24  iter: 22579  total_loss: 0.387  loss_cls: 0.258  loss_box_reg: 0.133  time: 0.3661  data_time: 0.0065  lr: 0.000100  max_mem: 2537M
[02/13 10:05:35 d2.utils.events]: eta: 0:45:17  iter: 22599  total_loss: 0.398  loss_cls: 0.271  loss_box_reg: 0.122  time: 0.3661  data_time: 0.0067  lr: 0.000100  max_mem: 2537M
[02/13 10:05:42 d2.utils.events]: eta: 0:45:09  iter: 22619  total_loss: 0.404  loss_cls: 0.276  loss_box_reg: 0.124  time: 0.3661  data_time: 0.0065  lr: 0.000100  max_mem: 2537M
[02/13 10:05:49 d2.utils.events]: eta: 0:45:03  iter: 22639  total_loss: 0.395  loss_cls: 0.276  loss_box_reg: 0.122  time: 0.3661  data_time: 0.0069  lr: 0.000100  max_mem: 2537M
[02/13 10:05:57 d2.utils.events]: eta: 0:44:55  i

In [17]:
cfg.OUTPUT_DIR

'tboard_logs/retinanet_test_2020-01-21T23:40/'

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [18]:
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")


cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.3
cfg.DATASETS.TEST = ("szprotawa_val", )
# cfg.TEST.SCORE_THRESHOLD = 0.3
# cfg.TEST.SCORE_THRESH_TEST = 0.3
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.3


cfg.TEST.DETECTIONS_PER_IMAGE = 10
predictor = DefaultPredictor(cfg)

predictor.cfg["TEST"]SOLVER.CHECKPOINT_PERIOD = 5000

SyntaxError: invalid syntax (<ipython-input-18-99a6d98dbb48>, line 17)

In [ ]:
1+2

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = DatasetCatalog.get("szprotawa_test")

In [ ]:
import time
for d in np.random.choice(dataset_dicts, 1):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1].copy(),
                   metadata=meta, 
                   scale=0.8)
    vis = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    f, a = plt.subplots(1,3, figsize=(21,7))
    a[0].imshow(im[:,:,::-1])
    a[1].imshow(vis.get_image())
    
    v = Visualizer(im[:,:,::-1].copy(),
                   metadata=meta, 
                   scale=0.8)
    vis = v.draw_dataset_dict(d)
    a[2].imshow(vis.get_image())

In [ ]:
_ = register_detectron2_datasets("szprott", "./data/szprotawa_patches_256/", 0.7, 0.15, 0.15, 100, {'test': splits['test'][:500]})

In [ ]:
from detectron2.evaluation import DatasetEvaluator, COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader



evaluator = COCOEvaluator("szprotawa_test", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "szprotawa_test")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

In [ ]:
1+2

In [ ]:
outputs.keys()